In [1]:
import json
import os
import glob

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series,DataFrame

#sklearn.cluster.
import pickle
import logging
from time import time
from gensim.models.callbacks import CallbackAny2Vec
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from scipy import stats
from sklearn.cluster import KMeans
from soyclustering import SphericalKMeans
#from sphere cluster import Spherical K Means
#from spherecluster import SphericalKMeans

In [2]:
class Callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 1
        self.training_loss = []

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 1:
            current_loss = loss
        else:
            current_loss = loss - self.loss_previous_step
        print(f"Loss after epoch {self.epoch}: {current_loss}")
        self.training_loss.append(current_loss)
        self.epoch += 1
        self.loss_previous_step = loss

In [3]:
f = open('./song2vec_filtered_model_train_epoch_100.dat', 'rb')
model=pickle.load(f)
#list = f.readlines()
#f.close()
print(model)

Word2Vec(vocab=341729, size=256, alpha=0.025)


In [4]:
model.most_similar('129204',topn=20)

C:\Users\woojung\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('239651', 0.9973549842834473),
 ('525280', 0.9889724254608154),
 ('541508', 0.9712550044059753),
 ('435340', 0.9097160696983337),
 ('214353', 0.8986788988113403),
 ('420356', 0.8914004564285278),
 ('264203', 0.8861410617828369),
 ('110380', 0.8835406303405762),
 ('373107', 0.8763835430145264),
 ('470836', 0.8757386207580566),
 ('237396', 0.874198317527771),
 ('37319', 0.8708698749542236),
 ('345194', 0.8698115348815918),
 ('626914', 0.8673614859580994),
 ('402002', 0.8641963601112366),
 ('633876', 0.8634134531021118),
 ('429931', 0.8627378940582275),
 ('199981', 0.8616964221000671),
 ('255742', 0.8595569133758545),
 ('690925', 0.8594090938568115)]

In [5]:
embedding_matrix = model.wv[model.wv.vocab.keys()]
embedding_matrix.shape

(341729, 256)

### KNN 모델 ->  Elbow를 사용하여 K 구하기 -> t-SNE 사용하여 256차월을 3D로 나눠서 visualize 하기

## KNN 모델 학습

In [6]:
class SphericalKmeans:
    """Spherical k-means clustering.

    Parameters
    ----------
    n_clusters : int, optional, default: 2
        Number of clusters to form
    init : numpy array or scipy sparse matrix, \
        shape (n_features, n_clusters), optional, default: None
        Initial column labels
    max_iter : int, optional, default: 20
        Maximum number of iterations
    n_init : int, optional, default: 1
        Number of time the algorithm will be run with different
        initializations. The final results will be the best output of `n_init`
        consecutive runs.
    random_state : integer or numpy.RandomState, optional
        The generator used to initialize the centers. If an integer is
        given, it fixes the seed. Defaults to the global numpy random
        number generator.
    tol : float, default: 1e-9
        Relative tolerance with regards to criterion to declare convergence
    weighting : boolean, default: True
        Flag to activate or deactivate TF-IDF weighting

    Attributes
    ----------
    labels_ : array-like, shape (n_rows,)
        cluster label of each row
    criterion : float
        criterion obtained from the best run
    criterions : list of floats
        sequence of criterion values during the best run
    """

    def __init__(self, n_clusters=2, init=None, max_iter=20, n_init=1,
                 tol=1e-9, random_state=None, weighting=True):
        self.n_clusters = n_clusters
        self.init = init
        self.max_iter = max_iter
        self.n_init = n_init
        self.tol = tol
        self.random_state = check_random_state(random_state)
        self.labels_ = None
        self.criterions = []
        self.criterion = -np.inf
        self.weighting = weighting
        self.Z = None
        self.Z_fuzzy = None

    def fit(self, X, y=None):
        """Perform clustering.

        Parameters
        ----------
        X : numpy array or scipy sparse matrix, shape=(n_samples, n_features)
            Matrix to be analyzed
        """

        check_array(X, pos=False)

        check_numbers_clustering(X, self.n_clusters)

        criterion = self.criterion

        if self.weighting:
            transformer = TfidfTransformer(norm='l2', smooth_idf=True)
            X = transformer.fit_transform(X)

        X = X.todense()
        X = np.array(X)
        X = sp.lil_matrix(X)
        #X = sp.csr_matrix(X)
        X = normalize(X)

        #X = X.astype(float)

        random_state = check_random_state(self.random_state)
        seeds = random_state.randint(np.iinfo(np.int32).max, size=self.n_init)
        for seed in seeds:
            print(" == New init == ")
            self.random_state = seed
            self._fit_single(X)
            # remember attributes corresponding to the best criterion
            if (self.criterion > criterion):
                criterion = self.criterion
                criterions = self.criterions
                labels_ = self.labels_
                z = self.Z
                z_fuzzy = self.Z_fuzzy

        self.random_state = random_state

        # update attributes
        self.criterion = criterion
        self.criterions = criterions
        self.row_labels_ = labels_
        self.Z = z
        self.Z_fuzzy = z_fuzzy


    def _fit_single(self, X, y=None):
        """Perform one run of clustering.

        Parameters
        ----------
        X : numpy array or scipy sparse matrix, shape=(n_samples, n_features)
            Matrix to be analyzed
        """
        K = self.n_clusters

        if self.init is None:
            Z = random_init_clustering(K, X.shape[0], self.random_state)
        else:
            Z = np.matrix(self.init, dtype=float)

        X = sp.lil_matrix(X)

        Z = sp.lil_matrix(Z)  # random_init function returns a nd_array

        change = True

        c_init = -np.inf
        c_list = []
        n_iter = 0

        while change and n_iter < self.max_iter:
            print("iteration:", n_iter)
            change = False

            # compute centroids (in fact only summation along cols)
            centers = Z.T*X  # centers = sparse matrix

            # normalize centroids
            centers = normalize(centers)

            # hard assignment
            #Z=centers*X.T
            Z1 = X * centers.T
            Z1 = Z1.todense()
            Z1 = np.array(Z1)
            Z = np.zeros_like(Z1)
            Z[np.arange(len(Z1)), Z1.argmax(1)] = 1
            Z = sp.csc_matrix(Z)

            # compute and check if cosine criterion still evolves
            k_times_k = Z.T * X * centers.T
            c = np.trace(k_times_k.todense())  # no trace for sp ...

            if np.abs(c - c_init) > 1e-9:
                c_init = c
                change = True
                c_list.append(c)
                print(c)
            n_iter += 1

        self.criterion = c
        self.criterions = c_list
        part = Z.todense().argmax(axis=1).tolist()
        self.labels_ = [item for sublist in part for item in sublist]
        self.Z = Z
        self.Z_fuzzy = Z1


In [7]:
kmean = SphericalKMeans(n_clusters=2, 
                        init=None, 
                        max_iter=20, 
                        #n_init=1, 
                        tol=1e-09, 
                        random_state=None)
                        #weighting=True)

In [8]:
kmeans = SphericalKMeans(#init='k-means++',
                         max_iter=300,
                         n_clusters=110,
                         #n_init=5,
                         #n_jobs=-1,
                         #normalize=True,
                         random_state=123,
                         #tol=0.0001,
                         verbose=0) 
#kmeans.fit(embedding_matrix)
#kmeans_df=pd.DataFrame(kmeans.cluster_centers_)
#.fit(X)
#kmeans.fit(embedding_matrix[0])
#kmeans_df = pd.DataFrame(kmeans.cluster_centers_, columns = ['Red', 'Green', 'Blue'])

In [9]:
spherical_kmeans = SphericalKMeans(
    n_clusters=1000,
    max_iter=10,
    verbose=1,
    init='k-means++',
    #sparsity='minimum_df',
    minimum_df_factor=0.05
)
from scipy.sparse import csr_matrix
csr = csr_matrix(embedding_matrix)

In [24]:
range_k_clusters = (0,341729)
kmeans_result = []

from scipy.sparse import csr_matrix
csr = csr_matrix(embedding_matrix)

for k in range(*range_k_clusters):
    # CLUSTERING
    spherical_kmeans = SphericalKMeans(
    n_clusters=k,
    max_iter=10,
    verbose=1,
    init='k-means++',
    #sparsity='minimum_df',
    minimum_df_factor=0.05)
    kmeans = kmeans.fit_predict(csr)
 
     # EVALUATE
     #WCSS = kmeans.inertia_
 
    kmeans_result.append(metric)
kmeans_result = pd.DataFrame(kmeans_result)
print(kmeans_result)

KeyboardInterrupt: 

In [20]:
labels = kmeans.fit_predict(csr)

In [ ]:
import pickle

f = open('./spherical_labels.dat', 'wb')
pickle.dump(labels, f)
f.close()

In [ ]:
f = open('./spherical_labels.dat', 'rb')
label=pickle.load(f)
#list = f.readlines()
#f.close()
print(label)

In [ ]:
len(labels) #train.data의 길이 341729  * 1 

In [44]:
label=pd.DataFrame(labels)
label.columns=["WCSS"]

In [ ]:
label
label.to_csv("WCSS.csv",mode='w')
dataset=pd.read_csv("WCSS.csv", index_col=0)
dataset.plot.bar()
plt.show()

In [ ]:
%matplotlib inline
#labb.plot.line()
#plt.show()
plt.plot(labb)
plt.plot(np.arrange(len(labb)),labb["WCSS"])

## 학습된 KNN 모델에서 Elbow를 구하여 K 구하기

In [49]:
import math
def locateOptimalElbow(x, y):
    # START AND FINAL POINTS
    p1 = (x[0], y[0])
    p2 = (x[-1], y[-1])
    
    # EQUATION OF LINE: y = mx + c
    m = (p2[1] - p1[1]) / (p2[0] - p1[0])
    c = (p2[1] - (m * p2[0]))
    # DISTANCE FROM EACH POINTS TO LINE mx - y + c = 0
    a, b = m, -1
    dist = np.array([abs(a*x0+b*y0+c)/math.sqrt(a**2+b**2) for x0, y0 in zip(x,y)])
    return np.argmax(dist) + x[0]

In [64]:
k_opt = locateOptimalElbow(label.index, label['WCSS'].values)
type(k_opt)
k_opt

# optimal_K 값을 'WCSS'의 Metric를 사용하여 계산
#skm_opt=pd.DataFrame({'skm_opt':k_opt})
#skm_opt=pd.concat([label, skm_opt],axis=1)
#skm_opt = label.loc[k_opt, "skm_object"]
#skm_opt
#최적의 K 구하기

307

In [ ]:
songs_cluster = songs.copy()
songs_cluster.loc[model.wv.vocab.keys(), 'cluster'] = skm_opt.labels_
songs_cluster['cluster'] = songs_cluster['cluster'].fillna(-1).astype('int').astype('category')
songs_cluster.head()

In [48]:
from sklearn.preprocessing import normalize

class KMeans:
    def __init__(self, n_clusters, max_iters):
        self.n_clusters = n_clusters
        self.max_iters = max_iters
        ...

    def fit(self, x):
        x = normalize(x, norm='l2')
        self.cluster_center_ = np.zeros((self.n_clusters, x.shape[1]))
        for iter_ in range(self.max_iters):
           ...
           self._centroid_update(x, labels)
           ...

    def _centroid_update(self, x, labels):
        for c in range(self.n_clusters):
            idxs = np.where(labels == c)[0]
            x_cluster = x[idxs,:]
            #k-means
            #self.cluster_center_[c,:] = x_cluster.sum(axis=0) / idxs.shape[0]
            
            #spherical k-means
            self.cluster_center_[c,:] = normalize(x_cluster.sum(axis=0), norm='l2')

In [ ]:
#알고리즘이 수렴하지 않는 문제가 발생할 수 있으므로, 이를 피하기 위하여 initialization method 를 "random"이 아닌 "k-means ++" 를 이용
#K값과 군집에 대한 해석은 분석자의 감에 의존할 수도 있음
##이를 방지하기 위하여 군집을 직접 "plotting" 해보거나 "Within Cluster Sum of Squared (Cluster inertia)"를 k값 별로 구하여 elbow point 찾는것에 도움이 됨


In [ ]:
def elbow(x,n_k): #sum of squared distance using sklearn
    sse=[]
    for i in range(1, n_k+1):
        km=KMeans(n_clusters=i, init="k-means++").fit(x)
        sse.append(km.inertia_)
    return  sse, range(1,n_k+1)
##plotting Within_cluster sum of suqares(=cluster inertia) to choose 'K'
def wcss(input, k):
    import seaborn as sns
    import matplotlib.pyplot as plt
    Info=elbow(input,k)
    plt.plot (Info[1], Info[0], marker='o', color='c')
    plt.xticks(Info[1])
    plt.xlabel ("K")
    plt.ylabel ("Total Squared Error")
    plt.title("Within #Cluster sum of squares")
    plt.show()


In [41]:
##clusture the data using the von-mises fisher distribution
#getting the size of the data
def sphericalknn (data,no_clusters):
    
    H=len(data)
    W=len(data[0])
    
#calculating the mean to remove the dc component
    sum_sample=sum(data)
    norm_sample=np.linalg.norm(sum_sample)
    mean_sample=(sum_sample)/(norm_sample)

#calculating global mean to get centroids of the clusters
    deviation=0.01
    mean_global = np.zeros([no_clusters,W])
    for i in range (0,no_clusters):
        random_sample=np.random.rand(1,W)-0.5
        random_norm=deviation*(np.random.rand())
        random_sample2=(random_norm*random_sample)/np.linalg.norm(random_sample)
        temp = mean_sample+random_sample2
        mean_global[i,:] = temp/np.linalg.norm(temp)

#print(mean_globalf)   
#calculating mean from spherical kmeans

    sum_sample3 = np.zeros([1,W])
    difference=1
    epsilon=0.01
    number=100
    iteration=0

    while (difference>epsilon):
        #check
        iteration=iteration+1
        number2=number
        #computing the nearest neighbour and assigning the points
        mean_global2 = np.transpose(mean_global)
        value=np.dot(data, mean_global2)
        value_max=value.max(1)
        clusters=np.argmax(value,axis=1)
    
    #computing value of the function
        number=sum(value_max) 
    #print(number)

    #computing centroids for the clusters
        for i in range(0,no_clusters):
            sum_sample3=sum(data[np.where(clusters==i)])
            if(mpmath.norm(sum_sample3) != 0):
                temp2=sum_sample3/mpmath.norm(sum_sample3)#np.linalg.norm(sum_sample3) #Check this
                mean_global[i,:] = temp2

        
        difference=abs(number-number2)
    
    return clusters

In [36]:
from sklearn import preprocessing  # to normalise existing X
X_Norm = preprocessing.normalize(embedding_matrix)

km2 = cluster.KMeans(n_clusters=5,init='random').fit(X_Norm)

NameError: name 'embedding_matrix' is not defined

In [32]:
def KMeans_cosine_fit(sparse_data, nclust = 10, njobs=-1, randomstate=None):
    # Manually override euclidean
    def euc_dist(X, Y = None, Y_norm_squared = None, squared = False):
        #return pairwise_distances(X, Y, metric = 'cosine', n_jobs = 10)
        return np.arccos(cosine_similarity(X, Y))/np.pi
    k_means_.euclidean_distances = euc_dist
    kmeans = k_means_.KMeans(n_clusters = nclust, n_jobs = njobs, random_state = randomstate)
    _ = kmeans.fit(sparse_data)
    return kmeans

In [34]:
kmeans=KMeans_cosine_fit(sparse_data, nclust=110, njobs=-1, randomstate=123)

''''spherical_kmeans = SphericalKMeans(n_init='k-means++', 
                                   max_iter=300, 
                                   n_clusters=110,
                                   n_init=5, 
                                   n_jobs=-1, 
                                   normalize=True, 
                                   random_state=123, 
                                   tol=0.0001, 
                                   verbose=0)

#labels = spherical_kmeans.fit_predict(x)'''

NameError: name 'sparse_data' is not defined

In [34]:
songs_cluster = songs.copy()
songs_cluster.loc[model.wv.vocab.keys(), 'cluster'] = skm_opt.labels_
songs_cluster['cluster'] = songs_cluster['cluster'].fillna(-1).astype('int').astype('category')
songs_cluster.head()

NameError: name 'songs' is not defined

In [12]:
k_opt = locateOptimalElbow(skm_df.index, skm_df['WCSS'].values)
skm_opt = skm_df.loc[k_opt, "skm_object"]
skm_opt

NameError: name 'locateOptimalElbow' is not defined

In [24]:
from sklearn.cluster import KMeans

In [26]:
for k in embedded_matrix:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(df)
    distortions.append(kmeanModel.inertia_)
kmeanModel=KMeans(n_clusters =3)
kmeanModel.fit(df)

NameError: name 'embedded_matrix' is not defined

In [33]:
from soyclustering import SphericalKMeans
spherical_kmeans = SphericalKMeans(init='k-means++', max_iter=300, n_cluster=110, n_init=5, n_jobs=-1, normalize =True, random_state=123,tol=0.0001, verbose=0)

labels = spherical_kmeans.fit_predict(x)

TypeError: __init__() got an unexpected keyword argument 'n_cluster'